# Concurrent.futures


컴퓨터 프로그램이 어떤 일을 처리해 나가는 실행 흐름은 기본적으로 선형이다.

하지만 두개 이상의 작업이 동시에 진행되어야 하는 상황이 필요할 수 있다.

이때 ‘동시성’이라는 것이 필요하게 되었고, 파이썬에서는 기본적으로 threading.Thread 와

Multiprocessing.Process 를 이용해서 다중 스레드 및 다중 프로세스를 통해서 동시성 작업을 처리했다.

python 3.2에서는 동시성에 관한 고수준 API 를 제공하게 되는데

이는 *concurrent.futures* 라이브러리이다.

In [ ]:
import threading


## Concurrent.futures 의 한계점과 장점

이 API 는 기존 멀티 스레드나 프로세스관련 API를 완전히 대체하지는 않는다.

concurrent 모듈은 기존 _thread 라이브러리에 의존하고 있다.

스레드나 프로세스를 저수준 API처럼 세세하게 제어하는 수단도 제공하지 않는다.

실제로 python reference 를 보면 그저 사용하는법 밖에 설명하고 있지 않다.

하지만 이를 상쇄하는 장점으로 Future 라는 개념이 도입됐다.

이로인해 분산처리 및 그외 동시성에 관련된 코드를 편리하게 작성할 수 있게 해준다

## Future

concurrent.futures 의 multi thread/process 모델에서는 데이터 처리의 흐름을 분산하는 것에 중점을 두었다.

- thread 와 process 의 흐름에 대해서 생각해보자

Future 클래스를 통해서 분산 처리된 결과를 손쉽게 원래의 스레드로 돌려줄 수 있다.

Future 란 별도의 실행 흐름에서 처리되는 코드를 객체화 한 것이다.

현재 Thread 는 작업을 스케줄링한 후 Future 객체를 얻게 된다.

이는 작업이 다른 어딘가에서 처리중이며 '미래의 어느 시점에 그 결과가 나오게 된다' 라는 약속을 의미한다.


In [3]:
from concurrent.futures import Future
print(Future)


<class 'concurrent.futures._base.Future'>


## Future 의 사용 방법

1. 특정 함수를 별도의 실행 흐름에서 시작하고, 해당 작업의 완료를 기다리면서 블럭되는 대신 Future 객체를 얻는다.

2. 위에서 시작하게 되면 Future 내의 작업은 아직 실행중이거나 혹은 결과는 생성되지 않다.

3. Future 객체 내의 완료 여부와 상관없이, 작업을 추가로 요청하거나, 취소할 수 있다.
  - 중요한 것은 현재의 스레드에서 다른 작업을 처리할 수도 있다.

4. 현재의 스레드에서 작업 결과가 필요한 시점이 오면 Future.result() 를 통해서 결과를 요청한다.
 - 이때 결과가 생성 O => 결과의 반환
 - 아직 처리중 = > result() 메소드를 실행한 시점부터 결과의 생성을 기다리도록 현재 스레드를 중지 가능



## Executor
위에서 말했다시피 사용자가 직접 Future 객체를 만들 필요가 없으며 이를 생성해주는 것이 Executor 이다.

Executor는 한정된 개수의 Thread / Process 를 가지게 된다.

Thread 를 사용하게 되면 ThreadPoolExecutor / process를 사용하게 되면 ProcessPoolExecutor를

사용하면 되며 동일한 코드에서 클래스만 선택해주면 어떤 환경인지 손쉽게 알 수 있다.


In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

# max_workers 의 max_workers= 프로세스갯수 * 5
with ThreadPoolExecutor(max_workers=25) as t_executor:
    t_executor.submit()
    t_executor.map()
    t_executor.shutdown()

# max_workers 의 max_workers= 시스템의 CPU 코어 갯수
with ProcessPoolExecutor(max_workers=5) as p_executor:
    p_executor.submit()
    p_executor.map()
    p_executor.shutdown()


In [ ]:
from concurrent.futures import ProcessPoolExecutor
def is_prime(n):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    for k in range(3, int(n**0.5 + 1.5), 2):
        if n % k == 0:
            return False
    return True

def load_url(url):
    conn = urlopen(url, timeout=3)
    return conn.read()


p_executor = ProcessPoolExecutor(max_workers=None)
num = 115797848077099

# 먼저 URL에 접속하여 페이지 데이터를 받는 것을 별도 프로세스에서 처리
f = p_executor.submit(load_url, 'http://google.co.kr')
print(f)

# 현재 프로세스는 소수 검사 로직을 처리
prime = is_prime(num)
print(f'{num} is prime: {prime}')

# 이제 페이지 데이터가 필요한 순간
data = f.result()
print(f'The page is {len(data)} bytes.')
p_executor.shutdown(True) # 작업을 종료합니다.


## .submit() - 개별작업 실행

```
t_executor = ThreadPoolExecutor()
t_executor.submit(work, *args, **kwargs)

with ThreadPoolExecutor as t_executor:
    # t_executor.submit(work, *args, **kwargs)
    # work(*arg, **kwargs)
    # Future 객체를 즉시 return 한다.
```


## .map() - 동시성 map 연산
```
with ThreadPoolExecutor as t_executor:
    t_executor.map(work, *args, **kwargs)
    all_result = list(t_executor.map(work, data, timeout=2))
    # timeout = seconds
```

기본함수 map()과 비슷하다

하지만 map() 과는 다르게 전달된 작업들을 가능한 빨리 가능하다면 동시에 시작된다.

모든 작업의 처리과 완료되면 변환된 결과가 리스트로 리턴된다.

즉, Future.result()가 반환 될때 이 결과를 취합하여 전달 해준다.

시간이 지난이후에도 결과가 나오지 않았다면 concurrent.futures.TimeoutError 가 발생한다.

In [2]:
from concurrent.futures import ProcessPoolExecutor

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]
def timer(f):
    def _():
        start_time = time.time()
        f()
        print(f"{f.__name__}() time --- {time.time() - start_time} seconds ---")
    return _

def is_prime(n):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    for k in range(3, int(n**0.5 + 1.5), 2):
        if n % k == 0:
            return False
    return True

@timer
def use_concurrent_process():
    with ProcessPoolExecutor() as p_executor:
        for num, prime in zip(PRIMES, p_executor.map(is_prime, PRIMES)):
            print(f'{num} is prime: {prime}')


@timer
def use_concurrent_thread():
    with ThreadPoolExecutor() as t_executor:
        for num, prime in zip(PRIMES, t_executor.map(is_prime, PRIMES)):
            print(f'{num} is prime: {prime}')

@timer
def dont_use_concurrent_process():
    for num, prime in zip(PRIMES, map(is_prime, PRIMES)):
            print(f'{num} is prime: {prime}')


if __name__ == '__main__':
    use_concurrent_process()
    print()
    print('==========================')
    print()
    use_concurrent_thread()
    print()
    print('==========================')
    print()
    dont_use_concurrent_process()

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
use_concurrent_process() time --- 0.9320831298828125 seconds ---


112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
use_concurrent_thread() time --- 2.3334009647369385 seconds ---


112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
dont_use_concurrent_process() time --- 2.3196940422058105 seconds ---


## .shutdown() - 리소스의 정리

executor 에게 종료 시그널을 보내며 더이상 사용하지 않음을 알린다.

시작하지않은 future 객체를 시작하지않고 취소,

진행중인 작업들은 종료될 때까지 기다린다.


## About Future

실행기의 .submit() 메소드를 사용하거나, 모듈 함수인 concurrent.futures.wait(),
 concurrent.futures.as_completed()를 사용하면 Future 객체를 얻게 된다.

cancel() : 작업 취소를 시도한다. 만약 현재 실행중이고 취소가 불가능할 경우 False를 리턴한다.
 작업이 취소되었다면 True가 리턴된다.

canceled() : 취소가 완료된 작업이면 True를 리턴한다.

running(): 실행 중인 경우 True를 리턴한다.

done(): 작업이 완료되어고 정상적으로 종료되었다면 True를 리턴한다.

result(timeout=): 결과를 대기한 후 리턴한다. timeout= 인자가 주어지면 해당 초까지 기다린다.

add_done_callback(): 완료 콜백을 추가할 수 있다. 콜백은 Future 를 인자로 받는 형태여야 하며,
future가 완료되거나 취소될 때 호출된다. (취소 여부를 알기 위해서는 콜백에 전달된 future에 대해서
.cancelled()를 호출해본다.) 콜백을 추가하는 시점에 이미 작업이 완료/취소되었다면 콜백이 즉시 호출된다.



## concurrent.futures module functions

concurrent.futures 모듈은 두가지 함수를 제공하는데 이들은 공통적으로 List[concurrent.futures.Future]
타입을 인자로 받는다.

즉 한개 이상의 Future 를 받아서 완료를 기다리거나 먼저 완료되는데로 하나씩 처리하는 것이다.

### concurrent.futures.wait()

wait() 함수는 asyncio.wait() 와 비슷하며 특정 조건까지 의 future의 시쿼스를 대기한다음 완료된 것들,
혹은 작동중인 것들으로 구성된 튜플을 리턴한다.

```
def wait(fs, timeout=None, return_when=ALL_COMPLETED):
    pass
```

리턴 시점에 대한 조건은 return_when 이 결정한다.
- FIRST_COMPLETD: 가장 빨리 끝난 하나의 작업이 있을때 리턴
- FIRST_EXCEPTION : 예외가 발생하면 리턴, 예외가 발생하지 않으면 모두 완료될 때 리턴
- ALL_COMPLETED : 기본값으로 모든 작업이 완료될 때 리턴.

timeout 은 주어진 시간후에 강제로 리턴할 수 있다.


### concurrent.futures.as_completed()

as_completed()함수는 이터레이터를 리턴하는데, 주로 for ... in 구문에 사용된다.

즉 주어진 future들 중에서 먼저 끝나는 것부터 차례로 순회하여 반복문을 돌리는 것이다.

executor.map() 은 주어진 연속열의 순서대로 결과를 리스트로 만들지만

concurrent.futures.as_completed()에서는 결과의 순서가 반드시 주어진 것이 아니며, 실제로 이터레이터는 결과가 아닌
각각의 future 객체를 리턴한다.



### as_completed() 예제

In [3]:
from concurrent.futures import ThreadPoolExecutor
from urllib.request import urlopen
import concurrent.futures
import json
import time

URLS = ['http://www.google.co.kr/',
        'http://www.daum.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']


def load_url(url):
    conn = urlopen(url, timeout=3)
    return conn.read()

@timer
def use_concurrent_pool():
    with ThreadPoolExecutor() as t_executor:
        ex_fs = {str(t_executor.submit(load_url, url)): url for url in URLS}
        print(json.dumps(ex_fs, indent=4))

        fs = {t_executor.submit(load_url, url): url for url in URLS}
        for f in concurrent.futures.as_completed(fs):
            try:
                url = fs[f]
                data = f.result()
                print(f'[{url}] has {len(data)} characters.')
            except Exception:
                print(f'Fail to open : {url}')

@timer
def use_concurrent_thread():
    with ThreadPoolExecutor() as t_executor:
        ex_fs = {str(t_executor.submit(load_url, url)): url for url in URLS}
        print(json.dumps(ex_fs, indent=4))

        fs = {t_executor.submit(load_url, url): url for url in URLS}
        for f in concurrent.futures.as_completed(fs):
            try:
                url = fs[f]
                data = f.result()
                print(f'[{url}] has {len(data)} characters.')
            except Exception:
                print(f'Fail to open : {url}')

@timer
def dont_use_concurrent():
    for url in URLS:
        try:
            data = load_url(url)
            print(f'[{url}] has {len(data)} characters.')
        except Exception:
                print(f'Fail to open : {url}')


if __name__ == '__main__':

    use_concurrent_pool()
    print()
    print('==========================')
    print()
    use_concurrent_thread()
    print()
    print('==========================')
    print()
    dont_use_concurrent()



{
    "<Future at 0x7f9767746ba8 state=running>": "http://www.google.co.kr/",
    "<Future at 0x7f9767746e10 state=running>": "http://www.daum.com/",
    "<Future at 0x7f9767746fd0 state=running>": "http://europe.wsj.com/",
    "<Future at 0x7f9767759198 state=running>": "http://www.bbc.co.uk/",
    "<Future at 0x7f9767759390 state=running>": "http://some-made-up-domain.com/"
}
Fail to open : http://europe.wsj.com/
[http://www.google.co.kr/] has 13346 characters.
Fail to open : http://www.daum.com/
Fail to open : http://some-made-up-domain.com/
Fail to open : http://www.bbc.co.uk/
use_concurrent_pool() time --- 1.630708932876587 seconds ---


{
    "<Future at 0x7f9767759198 state=running>": "http://www.google.co.kr/",
    "<Future at 0x7f97677855f8 state=running>": "http://www.daum.com/",
    "<Future at 0x7f976776dda0 state=running>": "http://europe.wsj.com/",
    "<Future at 0x7f976777d080 state=running>": "http://www.bbc.co.uk/",
    "<Future at 0x7f976776d048 state=running>": "htt